# Oficina 4 – Transações em Banco de Dados

Integrantes:
* André Luiz Girão Ferreira
* Sara
* Thaís
* Tiago

**Objetivo**: Experimentar estratégias para utilização de transações e níveis de isolamento em SGBDs relacionais. As tarefas envolvem uma simulação de um sistema de reservas de passagem áreas.








## Informações Iniciais - Simulação de Sistema de Reservas de Passagens

### Tabela e Atributos

Considere a seguinte tabela que registra os assentos reservados em um vôo:

***Assentos(num_voo,disp)***
onde

* ***num_voo***: é um número inteiro de 1 a 200
* ***disp***: é um atributo booleano cujo valor é **true** se o assento estiver vago e **false** caso contrário. **O valor inicial é *true***

### Reserva de Assento

A reserva de um assento é feita em três passos:

* **Passo 1:** O sistema recupera a lista dos assentos disponíveis.
* **Passo 2:** O cliente escolhe o assento. Esse passo deve ser simulado pela escolha aleatória de um dos assentos disponíveis, levando para isso um **tempo de escolha de 1 segundo**.
* **Passo 3:** O sistema registra a reserva do assento escolhido, **atualizando o valor de disp para false**.

Cada assento é reservado individualmente. Duas versões diferentes do processo de reserva devem ser implementadas:
* **Versão A:** A reserva é implementada como uma única transação que inclui os três passos acima.
* **Versão B:** A reserva inclui uma transação para o Passo 1 e outra para o Passo 3. O Passo 2 não faz parte das transações, mas deve ser executado.

### Os Agentes

Agentes de viagens são responsáveis por realizar as reservas de **200 clientes no total**. A atividade de um agente de viagens é simulada por uma ***thread***.

Experimentos devem ser realizados simulando a atuação de **k** agentes de viagem trabalhando simultaneamente, onde
* k = 1,2,4,6,8 e 10.

Cada agente/thread faz uma reserva de cada vez. **As threads devem ser reiniciadas até que todos os 200 clientes tenham seus assentos reservados**.

### Sobre os Experimentos

Dois conjuntos de experimentos devem ser feitos usando dois níveis de isolamento:  
* “read committed”; e
* “serializable”.  

Nos dois casos, o sistema deve ser configurado para **realizar bloqueios a nível de tupla** (linha).


### Preparação Inicial do Banco de Dados 

In [1]:
# instalando dependências
!pip install psycopg2

In [10]:
!pip uninstall psycopg2 psycopg2-binary
!pip install --no-cache-dir psycopg2-binary


In [11]:
# importando dependências
import threading
import random
import time
import psycopg2

In [26]:
# --- Configurações para conectar ao banco de dados padrão 'postgres' (para criar dbs) ---
DB_CONFIG_ADMIN = {
    'host': 'localhost',
    'port': '5432',
    'dbname': 'postgres',  # Conectar ao banco de dados padrão 'postgres'
    'user': 'postgres',
    'password': '1234',
    'options': '-c client_encoding=UTF8'
}

# --- Configurações para conectar ao seu banco de dados 'oficina4' ---
DB_CONFIG_OFICINA4 = {
    'host': 'localhost',
    'port': '5432',
    'dbname': 'oficina4',  # Seu novo banco de dados
    'user': 'postgres',
    'password': '1234',
    'options': '-c client_encoding=UTF8'
}

TOTAL_CLIENTES = 200

In [27]:
def get_conexao_db(options):
    return psycopg2.connect(**options)

In [36]:
def criar_banco_e_tabela():
    conn_admin = None
    conn_oficina4 = None

    try:
        # --- Parte 1: Conexão para criar o banco de dados (usando DB_CONFIG_ADMIN) ---
        conn_admin = get_conexao_db(DB_CONFIG_ADMIN)
        # É preferível usar set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT) para DDL
        conn_admin.autocommit = True
        cur_admin = conn_admin.cursor()

        # Cria o banco de dados "oficina4" se não existir
        cur_admin.execute("SELECT 1 FROM pg_database WHERE datname = 'oficina4'")
        existe = cur_admin.fetchone()
        if not existe:
            # Recomenda-se adicionar ENCODING, LC_COLLATE e TEMPLATE para robustez
            cur_admin.execute("CREATE DATABASE oficina4 WITH ENCODING 'UTF8' LC_COLLATE 'C' LC_CTYPE 'C' TEMPLATE template0;")
            print("Banco de dados 'oficina4' criado com sucesso.")
        else:
            print("Banco de dados 'oficina4' já existe.")

        cur_admin.close()
        conn_admin.close() # FECHE a conexão administrativa aqui!

        # --- Parte 2: Conexão ao banco recém-criado "oficina4" para criar a tabela ---
        print("Tentando conectar ao banco de dados 'oficina4' para criar a tabela...")
        conn_oficina4 = get_conexao_db(DB_CONFIG_OFICINA4)
        conn_oficina4.autocommit = True
        # Para DDL (CREATE TABLE), também é bom usar autocommit ou set_isolation_level
        # Se você for fazer operações DML (INSERT, UPDATE) depois, considere desativar autocommit
        # e usar conn.commit() para essas operações.
        cur_oficina4 = conn_oficina4.cursor()

        # Cria a tabela Assentos se não existir
        cur_oficina4.execute("""
            CREATE TABLE IF NOT EXISTS Assentos (
                num_voo INTEGER PRIMARY KEY CHECK (num_voo BETWEEN 1 AND 200),
                disp BOOLEAN DEFAULT TRUE
            );
        """)
        # Com autocommit ativado, conn.commit() aqui é redundante, mas não prejudicial.
        # Se você tivesse set_isolation_level(psycopg2.IsolationLevel.READ_COMMITTED)
        # ou outro nível padrão, o commit seria necessário.
        # Vou deixar o commit para mostrar que em cenários de DML ele seria importante.
        # conn_oficina4.commit() # Remova esta linha se autocommit estiver sempre ativo para DDL.
        print("Tabela 'Assentos' criada com sucesso.")

        cur_oficina4.close()

    except psycopg2.Error as e: # Capture exceções mais específicas de psycopg2
        print(f"Erro ao criar banco ou tabela: {e}")
    except Exception as e: # Capture outras exceções genéricas
        print(f"Ocorreu um erro inesperado: {e}")
    finally:
        # Garanta que ambas as conexões sejam fechadas se estiverem abertas
        if conn_admin is not None:
            conn_admin.close()
        if conn_oficina4 is not None:
            conn_oficina4.close()

In [29]:
# --- Inicializa os assentos (roda uma vez) ---
def inicializar_assentos():
    conn = get_conexao_db(DB_CONFIG_OFICINA4)
    cur = conn.cursor()
    cur.execute("DELETE FROM Assentos;")  # limpa a tabela
    for i in range(1, 201):
        cur.execute("INSERT INTO Assentos (num_voo, disp) VALUES (%s, TRUE);", (i,))
    conn.commit()
    cur.close()
    conn.close()

In [37]:
criar_banco_e_tabela()

Banco de dados 'oficina4' já existe.
Tentando conectar ao banco de dados 'oficina4' para criar a tabela...
Tabela 'Assentos' criada com sucesso.


In [38]:
inicializar_assentos()

### Tarefa 1

Implemente as **versões A e B do processo de reserva**. É importante que as implementações tratem adequadamente conflitos de Concorrência como **deadlocks e rollbacks**


In [39]:
# --- Versão A: tudo em uma transação ---
def reservar_assento_versao_a(id_agente):
    try:
        conn = get_conexao_db()
        cur = conn.cursor()
        conn.autocommit = False  # transação manual

        # Passo 1: buscar assentos disponíveis
        cur.execute("SELECT num_voo FROM Assentos WHERE disp = TRUE FOR UPDATE;")
        disponiveis = cur.fetchall()

        if not disponiveis:
            print(f"[Agente-{id_agente}]: Nenhum assento disponível.")
            conn.rollback()
            return

        # Passo 2: escolher assento (fora do BD)
        time.sleep(1)
        escolhido = random.choice(disponiveis)[0]

        # Passo 3: reservar
        cur.execute("UPDATE Assentos SET disp = FALSE WHERE num_voo = %s;", (escolhido,))
        conn.commit()
        print(f"[Agente-{id_agente}]: Reservado assento {escolhido}")

    except Exception as e:
        print(f"[Agente-{id_agente}]: Erro: {e}")
        conn.rollback()
    finally:
        cur.close()
        conn.close()

In [40]:
# --- Versão B: duas transações ---
def reservar_assento_versao_b(id_agente):
    try:
        conn = get_conexao_db()
        conn.autocommit = False
        
        # Transação 1: buscar disponíveis
        cur1 = conn.cursor()
        
        cur1.execute("SELECT num_voo FROM Assentos WHERE disp = TRUE;")
        disponiveis = cur1.fetchall()
        
        if not disponiveis:
            print(f"[Agente-{id_agente}]: Nenhum assento disponível.")
            conn.rollback()
            return

        conn.commit()
        cur1.close()
        
        # Passo 2: escolher (tempo de escolha)
        time.sleep(1)
        escolhido = random.choice(disponiveis)[0]

        # Transação 2: tentativa de reserva
        cur2 = conn.cursor()
        cur2.execute("UPDATE Assentos SET disp = FALSE WHERE num_voo = %s AND disp = TRUE;", (escolhido,))
        
        if cur2.rowcount == 0:
            print(f"[Agente-{id_agente}]: Assento {escolhido} já foi reservado por outro agente.")
            conn.rollback()
        else:
            conn.commit()
            print(f"[Agente-{id_agente}]: Reservado assento {escolhido}")

    except Exception as e:
        print(f"[Agente-{id_agente}]: Erro: {e}")
        conn.rollback()
    finally:
        if 'cur1' in locals(): cur1.close()
        if 'cur2' in locals(): cur2.close()
        if 'conn' in locals(): conn.close()

In [41]:
# --- Gerenciador de threads ---
def executar_reservas(versao, num_agentes):
    agentes = []
    reservar_assento = reservar_assento_versao_a if versao == "A" else reservar_assento_versao_b

    #criando threads
    for i in range(num_agentes):
        id_agente = i+1
        t = threading.Thread(target=reservar_assento, args=(id_agente,))
        agentes.append(t)
        t.start()

    #esperando threads acabarem
    for agente in agentes:
        agente.join()

    print(f"--- Reservas versão {versao} finalizadas ---")

In [ ]:
numero_agentes = [1,2,4,6,8,10]  #numero k de agentes

for k in numero_agentes:
    executar_reservas(versao="A", num_agentes=numero_agentes)

for k in numero_agentes:
    executar_reservas(versao="B", num_agentes=numero_agentes)

### Tarefa 2

Apresente gráficos de linha onde, para cada valor de k (número de agentes) no eixo x, temos no eixo y o tempo necessário para que todos os clientes efetuem suas reservas. Um gráfico diferente deve ser apresentado para cada par de versões da reserva e nível de isolamento.



### Tarefa 3

Apresente uma tabela com o número máximo, mínimo e médio de vezes que um cliente teve que tentar reservar um assento até conseguir, ou seja, o número de vezes que uma reserva teve que ser refeita. A tabela considera as variações de k, versão de reserva e nível de isolamento.


### Tarefa 4

Apresente uma análise dos resultados obtidos em cada versão de reserva e tipo de isolamento, explicando as diferenças entre resultados.


### Tarefa 5

Análise de Conflitos de Concorrência. Para cada experimento executado, registre o número de deadlocks e rollbacks detectados pelo sistema gerenciador de banco de dados.

**O que entregar:** Um resumo tabular com o número total de deadlocks e rollbacks para cada combinação de (versão de reserva, nível de isolamento, valor de k). Indicação de como os erros foram tratados no código (ex: tentativas de reexecução, logs etc.).

### Tarefa 6 - Avaliação de Variação na Ordem de Alocação de Assentos

Compare a ordem final dos assentos ocupados (i.e., a sequência de num_voo com disp = false) entre diferentes execuções de um mesmo cenário com concorrência (mesmo k, versão e nível de isolamento), para identificar variações causadas por condições de corrida.

**O que entregar:** Para cada combinação de parâmetros com k > 1, execute o experimento 3 vezes e apresente a variação na ordem dos assentos alocados; Discuta a relação dessa variação com o nível de isolamento e a estrutura transacional usada.



### Tarefa 7 - Demonstração de Anomalias de Concorrência em Diferentes Níveis de Isolamento

O objetivo desta tarefa é observar experimentalmente a ocorrência (ou não) de três tipos clássicos de anomalias em transações concorrentes, sob os níveis de isolamento **READ COMMITTED** e **SERIALIZABLE**. Para isso, implemente três experimentos distintos e controlados, cada um projetado para testar um fenômeno. Cada experimento deve ser executado duas vezes, uma com cada nível de isolamento, e a diferença de comportamento deve ser registrada.



## Experimentos

### Experimento A - Non-repeatable Read

**Descrição:** Uma transação lê um mesmo dado duas vezes, mas entre essas duas leituras, outra transação modifica o dado e realiza commit.

**Cenário:**
* T1 inicia e lê o valor de um assento específico.
* T2 inicia, atualiza o mesmo assento para reservado (disp=false) e comita.
* T1 tenta reler o mesmo assento.

**O que entregar:**
* Código das transações T1 e T2.
* Logs ou saídas indicando se T1 leu valores diferentes na primeira e segunda leitura.
* Análise: isso ocorreu em qual nível de isolamento?




### Experimento B - Phanton Read

**Descrição:** Uma transação executa a mesma consulta duas vezes e obtém conjuntos de resultados diferentes porque outra transação inseriu ou removeu dados que se encaixam no critério da consulta.

**Cenário:**
* T1 inicia e consulta todos os assentos vagos (disp=true).
* T2 inicia, insere um novo assento vago (disp=true) ou atualiza um assento para vago, e comita.
* T1 executa novamente a mesma consulta.


**O que entregar:**
* Código de T1 e T2.
* Saída das duas execuções da consulta de T1.
* Indicação se houve alteração no resultado.
* Discussão da relação com o nível de isolamento usado.


### Experimento C - Dirty Read

**Descrição:** Uma transação lê dados modificados por outra transação que ainda não fez commit (ou que foi revertida). Esse experimento serve para mostrar que o PostgreSQL não permite dirty reads nem mesmo em READ COMMITTED, então o comportamento esperado é que não ocorra a leitura suja.

**Cenário:**
* T1 inicia e atualiza o valor de um assento (disp=false), mas não comita.
* T2 inicia e tenta ler o mesmo assento.
* T1 faz rollback.


**O que entregar:**
* Código de T1 e T2 com sincronização apropriada para simular esse cenário.
* Comprovação de que T2 não teve acesso ao valor não confirmado.
* Discussão: por que isso ocorre mesmo em READ COMMITTED?